## Load Packages

In [1]:
include("printmat.jl")

println4Ps (generic function with 1 method)

In [2]:
using Plots
backend = "gr"              #"gr" (default), "pyplot" 

if backend == "pyplot"
    pyplot(size=(600,400))
else    
    gr(size=(600,400))
end

Plots.GRBackend()

# Mean-Variance: Brute Force

In [3]:
μ = [0.115; 0.095; 0.06]        #expected returns
Σ  = [166  34  58;              #covariance matrix
       34  64   4;
       58   4 100]/100^2                  

println("expected returns: ")
printmat(μ)
println("covariance matrix:")
printmat(Σ)

expected returns: 
     0.115
     0.095
     0.060

covariance matrix:
     0.017     0.003     0.006
     0.003     0.006     0.000
     0.006     0.000     0.010



To calculate the expeted return and the variance of a portfolio, use $ER_p = w'\mu$ and $Var(R_p) = w'\Sigma w$.

In [4]:
w = [ 0    1    0   ;
      0.22 0.30 0.48;
      0.02 0.63 0.35;
      0.25 0.68 0.07]'        #4 different portfolios (try many...), notice '

println("Several different portfolios (one in each column): ")
printmat(w)

n = size(w,2)
ERp   = fill(NaN,n)
StdRp = fill(NaN,n)
for i = 1:n
    ERp[i]   = (w[:,i]'μ)[1]
    StdRp[i] = sqrt((w[:,i]'Σ*w[:,i])[1])
end    
println("expected portfolio return and Std of portfolio return for $n portfolios, %: ")
printmat([ERp StdRp]*100)

Several different portfolios (one in each column): 
     0.000     0.220     0.020     0.250
     1.000     0.300     0.630     0.680
     0.000     0.480     0.350     0.070

expected portfolio return and Std of portfolio return for 4 portfolios, %: 
     9.500     8.000
     8.260     7.398
     8.315     6.415
     9.755     7.378



In [5]:
plot1 = scatter(StdRp*100,ERp*100,legend=nothing,color=:red,xlims=(0,15),ylims=(0,15))
title!("Mean vs standard deviation")
xlabel!("Std(Rp), %")
ylabel!("ERp, %")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.0 
 
 
 2.5 
 
 
 5.0 
 
 
 7.5 
 
 
 10.0 
 
 
 12.5 
 
 
 15.0 
 
 
 0.0 
 
 
 2.5 
 
 
 5.0 
 
 
 7.5 
 
 
 10.0 
 
 
 12.5 
 
 
 15.0 
 
 
 Mean vs standard deviation 
 
 
 Std(Rp), % 
 
 
 ERp, %

# MV Frontier with Two Assets

Recall: with only two investable assets, all portfolios of them are on the MV frontier

In [6]:
μ = [0.115; 0.06]            #means
Σ  = [166   58;              #covariance matrix
       58  100]/100^2                      

w1 = collect(1:-0.05:0)            #weight on asset 1 
n  = length(w1)
ERp   = fill(NaN,n)
StdRp = fill(NaN,n)
for i = 1:n
    w        = [w1[i];1-w1[i]]
    ERp[i]   = (w'μ)[1]
    StdRp[i] = sqrt((w'Σ*w)[1])
end   

plot2 = plot(StdRp*100,ERp*100,legend=nothing,color=:red,xlims=(0,15),ylims=(0,15))
scatter!(sqrt.(diag(Σ))*100,μ*100)
title!("Mean vs standard deviation")
xlabel!("Std(Rp), %")
ylabel!("ERp, %")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.0 
 
 
 2.5 
 
 
 5.0 
 
 
 7.5 
 
 
 10.0 
 
 
 12.5 
 
 
 15.0 
 
 
 0.0 
 
 
 2.5 
 
 
 5.0 
 
 
 7.5 
 
 
 10.0 
 
 
 12.5 
 
 
 15.0 
 
 
 Mean vs standard deviation 
 
 
 Std(Rp), % 
 
 
 ERp, %

# Calculating the MV Frontier (of only Risky Assets)

All portfolio on the MV frontier of risky assets only have (vectors of) portfolio weights as in

$w = \Sigma^{-1}(\mu \lambda + \mathbf{1} \delta)$, 

where $\lambda$ and $\delta$ depends on $(\mu,\Sigma)$ and the required average return $\mu^*$

In [7]:
μ = [0.115; 0.095; 0.06]    #expected returns
Σ  = [166  34  58;              #covariance matrix
       34  64   4;
       58   4 100]/100^2                  
Rf = 0.03

println("μ, Σ and Rf: ")
printmat(μ)
printmat(Σ)
printmat(Rf)

μ, Σ and Rf: 
     0.115
     0.095
     0.060

     0.017     0.003     0.006
     0.003     0.006     0.000
     0.006     0.000     0.010

     0.030



Some of the intermediate calculations are as follows. (Later on we put it all in a function)

In [8]:
mustar = 0.1                          #required average return  

n    = length(μ)
oneV = ones(n,1) 
 
Σ_1  = inv(Σ)

println("inv(Σ):")
printmat(Σ_1)
println("Σ*inv(Σ):")
printmat(Σ*Σ_1)

A    = (μ'Σ_1*μ)[1]                  #()[1] to make it a scalar, not an 1x1 matrix
B    = (μ'Σ_1*oneV)[1]
C    = (oneV'Σ_1*oneV)[1]
printlnPs("A, B, C: ",[A B C ])

λ    = (C*mustar - B)/(A*C-B^2)
δ    = (A-B*mustar)/(A*C-B^2)
printlnPs("λ and δ: ",[λ δ])

w    = Σ_1 *(μ*λ + oneV*δ)
println("w: ")
printmat(w)

inv(Σ):
    85.735   -42.545   -48.024
   -42.545   177.754    17.566
   -48.024    17.566   127.151

Σ*inv(Σ):
     1.000     0.000     0.000
     0.000     1.000     0.000
    -0.000     0.000     1.000

A, B, C:      1.804    19.759   244.634
λ and δ:      0.093    -0.003
w: 
     0.288
     0.690
     0.022



In [9]:
function MVCalc(mustar,μ,Σ)           #calculates the std of a portfolio on MVF of risky assets
    n    = length(μ)
    oneV = ones(n,1) 
    Σ_1  = inv(Σ)
    A    = (μ'Σ_1*μ)[1]                  #()[1] to make it a scalar, not an 1x1 matrix
    B    = (μ'Σ_1*oneV)[1]
    C    = (oneV'Σ_1*oneV)[1]
    λ    = (C*mustar - B)/(A*C-B^2)
    δ    = (A-B*mustar)/(A*C-B^2)
    w    = Σ_1 *(μ*λ + oneV*δ)
    StdRp = sqrt((w'Σ*w)[1])
    return StdRp,w
end

MVCalc (generic function with 1 method)

In [10]:
(StdAt10,wAt10) = MVCalc(0.1,μ,Σ)
println("Just testing: std and w of portfolio with mean return 10: ", round(StdAt10,3))
printmat(wAt10)

Just testing: std and w of portfolio with mean return 10: 0.077
     0.288
     0.690
     0.022



In [11]:
mustar = linspace(Rf,0.15,101)
L      = length(mustar)
StdRp  = fill(NaN,L)
for i = 1:L
    StdRP_i, = MVCalc(mustar[i],μ,Σ)             #, = means skipping the remaining output
    StdRp[i] = StdRP_i
end    

plot3 = plot(StdRp*100,mustar*100,legend=nothing,color=:red,xlims=(0,15),ylims=(0,15))
scatter!(sqrt.(diag(Σ))*100,μ*100)
title!("Mean vs standard deviation")
xlabel!("Std(Rp), %")
ylabel!("ERp, %")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.0 
 
 
 2.5 
 
 
 5.0 
 
 
 7.5 
 
 
 10.0 
 
 
 12.5 
 
 
 15.0 
 
 
 0.0 
 
 
 2.5 
 
 
 5.0 
 
 
 7.5 
 
 
 10.0 
 
 
 12.5 
 
 
 15.0 
 
 
 Mean vs standard deviation 
 
 
 Std(Rp), % 
 
 
 ERp, % 
 
<polyline clip-path="url(#clip02)" style="stroke:#ff0000; stroke-width:1; stroke-opacity:1; fill:none" points="
 512.649,296.378 504.26,293.493 495.923,290.608 487.641,287.723 479.417,284.838 471.254,281.953 463.156,279.068 455.127,276.183 447.17,273.298 439.291,270.413 
 431.493,267.528 423.782,264.643 416.163,261.757 408.642,258.872 401.225,255.987 393.918,253.102 386.728,250.217 379.662,247.332 372.73,244.447 365.938,241.562 
 359.297,238.677 352.815,235.792 346.503,232.907 340.371,230.022 334.431,227.137 328.694,224.252 323.174,221.367 317.881,218.482 312.831,215.597 308.036,212.712 
 303.51,209.827 299.268,206.942 295.324,204.057 291.691,201.172 288.384,198.287 285.414,195.402 282.796,192.517 280.539,189.631 278.655,186.746 277.152,183.861 
 276.037,180.976 275.317,178.091 274.994,175.206 275.07,172.321 275.545,169.436 276.417,166.551 277.681,163.666 279.33,160.781 281.358,157.896 283.754,155.011 
 286.508,152.126 289.608,149.241 293.042,146.356 296.795,143.471 300.855,140.586 305.207,137.701 309.837,134.816 314.731,131.931 319.875,129.046 325.257,126.161 
 330.862,123.276 336.677,120.391 342.692,117.506 348.894,114.62 355.272,111.735 361.816,108.85 368.516,105.965 375.362,103.08 382.347,100.195 389.46,97.3102 
 396.696,94.4252 404.046,91.5402 411.504,88.6551 419.063,85.7701 426.718,82.885 434.463,80 442.292,77.115 450.202,74.2299 458.186,71.3449 466.242,68.4598 
 474.365,65.5748 482.552,62.6898 490.798,59.8047 499.102,56.9197 507.459,54.0346 515.867,51.1496 524.323,48.2646 532.826,45.3795 541.372,42.4945 549.959,39.6094 
 558.585,36.7244 567.249,33.8394 575.948,30.9543 584.681,28.0693 593.446,25.1843 602.242,22.2992 611.067,19.4142 619.921,16.5291 628.801,13.6441 637.706,10.7591 
 646.636,7.87402 
 "/>

# Calculating the MV Frontier (of Risky and Riskfree Assets)

All portfolios on the MV frontier of both risky and riskfree have (vectors of) portfolio weights on the risky assets as in 

$w=\frac{\mu^{\ast}-R_{f}}{(\mu^{e})^{\prime}\Sigma^{-1}\mu^{e}}\Sigma^{-1}
\mu^{e}$, 

where $\mu^*$ is the required average return.

The weight of the riskfree asset is $1-\mathbf{1}'w$

In [12]:
function MVCalcRf(mustar,μ,Σ,Rf)           #calculates the std of a portfolio on MVF of (Risky,Riskfree)
    n     = length(μ)
    μe    = μ - Rf                         #expected excess returns            
    Σ_1   = inv(Σ)
    w     = (mustar-Rf)/(μe'Σ_1*μe)[1] * Σ_1 *μe
    StdRp = sqrt((w'Σ*w)[1])
    return StdRp,w
end

MVCalcRf (generic function with 1 method)

In [13]:
StdRpRf  = fill(NaN,L)
for i = 1:L
    StdRpRf_i, = MVCalcRf(mustar[i],μ,Σ,Rf) 
    StdRpRf[i] = StdRpRf_i
end    

plot4 = plot(StdRp*100,mustar*100,legend=nothing,color=:red,xlims=(0,15),ylims=(0,15))
plot!(StdRpRf*100,mustar*100,legend=nothing,color=:blue)
scatter!(sqrt.(diag(Σ))*100,μ*100)
title!("Mean vs standard deviation")
xlabel!("Std(Rp), %")
ylabel!("ERp, %")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.0 
 
 
 2.5 
 
 
 5.0 
 
 
 7.5 
 
 
 10.0 
 
 
 12.5 
 
 
 15.0 
 
 
 0.0 
 
 
 2.5 
 
 
 5.0 
 
 
 7.5 
 
 
 10.0 
 
 
 12.5 
 
 
 15.0 
 
 
 Mean vs standard deviation 
 
 
 Std(Rp), % 
 
 
 ERp, % 
 
<polyline clip-path="url(#clip02)" style="stroke:#ff0000; stroke-width:1; stroke-opacity:1; fill:none" points="
 512.649,296.378 504.26,293.493 495.923,290.608 487.641,287.723 479.417,284.838 471.254,281.953 463.156,279.068 455.127,276.183 447.17,273.298 439.291,270.413 
 431.493,267.528 423.782,264.643 416.163,261.757 408.642,258.872 401.225,255.987 393.918,253.102 386.728,250.217 379.662,247.332 372.73,244.447 365.938,241.562 
 359.297,238.677 352.815,235.792 346.503,232.907 340.371,230.022 334.431,227.137 328.694,224.252 323.174,221.367 317.881,218.482 312.831,215.597 308.036,212.712 
 303.51,209.827 299.268,206.942 295.324,204.057 291.691,201.172 288.384,198.287 285.414,195.402 282.796,192.517 280.539,189.631 278.655,186.746 277.152,183.861 
 276.037,180.976 275.317,178.091 274.994,175.206 275.07,172.321 275.545,169.436 276.417,166.551 277.681,163.666 279.33,160.781 281.358,157.896 283.754,155.011 
 286.508,152.126 289.608,149.241 293.042,146.356 296.795,143.471 300.855,140.586 305.207,137.701 309.837,134.816 314.731,131.931 319.875,129.046 325.257,126.161 
 330.862,123.276 336.677,120.391 342.692,117.506 348.894,114.62 355.272,111.735 361.816,108.85 368.516,105.965 375.362,103.08 382.347,100.195 389.46,97.3102 
 396.696,94.4252 404.046,91.5402 411.504,88.6551 419.063,85.7701 426.718,82.885 434.463,80 442.292,77.115 450.202,74.2299 458.186,71.3449 466.242,68.4598 
 474.365,65.5748 482.552,62.6898 490.798,59.8047 499.102,56.9197 507.459,54.0346 515.867,51.1496 524.323,48.2646 532.826,45.3795 541.372,42.4945 549.959,39.6094 
 558.585,36.7244 567.249,33.8394 575.948,30.9543 584.681,28.0693 593.446,25.1843 602.242,22.2992 611.067,19.4142 619.921,16.5291 628.801,13.6441 637.706,10.7591 
 646.636,7.87402 
 "/>
<polyline clip-path="url(#clip02)" style="stroke:#0000ff; stroke-width:1; stroke-opacity:1; fill:none" points="
 39.3701,296.378 44.1997,293.493 49.0293,290.608 53.8589,287.723 58.6886,284.838 63.5182,281.953 68.3478,279.068 73.1774,276.183 78.007,273.298 82.8367,270.413 
 87.6663,267.528 92.4959,264.643 97.3255,261.757 102.155,258.872 106.985,255.987 111.814,253.102 116.644,250.217 121.474,247.332 126.303,244.447 131.133,241.562 
 135.962,238.677 140.792,235.792 145.622,232.907 150.451,230.022 155.281,227.137 160.111,224.252 164.94,221.367 169.77,218.482 174.599,215.597 179.429,212.712 
 184.259,209.827 189.088,206.942 193.918,204.057 198.748,201.172 203.577,198.287 208.407,195.402 213.236,192.517 218.066,189.631 222.896,186.746 227.725,183.861 
 232.555,180.976 237.385,178.091 242.214,175.206 247.044,172.321 251.873,169.436 256.703,166.551 261.533,163.666 266.362,160.781 271.192,157.896 276.021,155.011 
 280.851,152.126 285.681,149.241 290.51,146.356 295.34,143.471 300.17,140.586 304.999,137.701 309.829,134.816 314.658,131.931 319.488,129.046 324.318,126.161 
 329.147,123.276 333.977,120.391 338.807,117.506 343.636,114.62 348.466,111.735 353.295,108.85 358.125,105.965 362.955,103.08 367.784,100.195 372.614,97.3102 
 377.443,94.4252 382.273,91.5402 387.103,88.6551 391.932,85.7701 396.762,82.885 401.592,80 406.421,77.115 411.251,74.2299 416.08,71.3449 420.91,68.4598 
 425.74,65.5748 430.569,62.6898 435.399,59.8047 440.229,56.9197 445.058,54.0346 449.888,51.1496 454.717,48.2646 459.547,45.3795 464.377,42.4945 469.206,39.6094 
 474.036,36.7244 478.866,33.8394 483.695,30.9543 488.525,28.0693 493.354,25.1843 498.184,22.2992 503.014,19.4142 507.843,16.5291 512.673,13.6441 517.502,10.7591 
 522.332,7.87402 
 "/>

# Tangency Portfolio

The tangency portfolio is a particular portfolio on the MV frontier of risky and riskfree, where the weights on the risky assets sum to one. It is therefore also on the MV frontier of risky assets only. The vector of portfolio weights is

$w_{T}=\frac{\Sigma^{-1}\mu^{e}}{\mathbf{1}^{\prime}\Sigma^{-1}\mu^{e}}$

In [14]:
function MVTangencyP(μ,Σ,Rf)           #calculates the tangency portfolio
    n     = length(μ)
    oneV  = ones(n,1)  
    μe    = μ - Rf                     #expected excess returns            
    Σ_1   = inv(Σ)
    w     = Σ_1 *μe/(oneV'Σ_1*μe)[1]
    muT   = (w'μ + (1-sum(w))*Rf)[1]
    StdT = sqrt((w'Σ*w)[1])
    return w,muT,StdT
end

MVTangencyP (generic function with 1 method)

In [15]:
(wT,muT,StdT) = MVTangencyP(μ,Σ,Rf)
println("Tangency portfolio: ")
printmat(wT)
printlnPs("mean and std of tangency portfolio, %: ",[muT StdT]*100)

Tangency portfolio: 
     0.248
     0.682
     0.070

mean and std of tangency portfolio, %:      9.750     7.372


By mixing the tangency portfolio and the riskfree, we can create any point on the MV frontier (for risky and riskfree). 

The code below shows the expected return and standard deviation of several portfolio (different $v$ values) of the form

$R_p = v w_T'R + (1-v)R_f$ 

In [16]:
v = [0;0.44;1;1.41]                                 #try different mixes of wT and Rf

(ERLev,StdLev)  = (similar(v),similar(v))           #to store results in
for i = 1:length(v)
    w          = v[i]*wT                          #risky portfolio
    ERLev[i]   = (w'μ + (1-v[i])*Rf)[1]     #portfolio with v[i] in wT and 1-v[i] in Rf
    StdLev[i]  = sqrt((w'Σ*w)[1])
end


plot(StdRp*100,mustar*100,legend=nothing,color=:red,xlims=(0,15),ylims=(0,15))
plot!(StdRpRf*100,mustar*100,legend=nothing,color=:blue)
scatter!(StdLev*100,ERLev*100)
title!("Mean vs standard deviation")
xlabel!("Std(Rp), %")
ylabel!("ERp, %")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.0 
 
 
 2.5 
 
 
 5.0 
 
 
 7.5 
 
 
 10.0 
 
 
 12.5 
 
 
 15.0 
 
 
 0.0 
 
 
 2.5 
 
 
 5.0 
 
 
 7.5 
 
 
 10.0 
 
 
 12.5 
 
 
 15.0 
 
 
 Mean vs standard deviation 
 
 
 Std(Rp), % 
 
 
 ERp, % 
 
<polyline clip-path="url(#clip02)" style="stroke:#ff0000; stroke-width:1; stroke-opacity:1; fill:none" points="
 512.649,296.378 504.26,293.493 495.923,290.608 487.641,287.723 479.417,284.838 471.254,281.953 463.156,279.068 455.127,276.183 447.17,273.298 439.291,270.413 
 431.493,267.528 423.782,264.643 416.163,261.757 408.642,258.872 401.225,255.987 393.918,253.102 386.728,250.217 379.662,247.332 372.73,244.447 365.938,241.562 
 359.297,238.677 352.815,235.792 346.503,232.907 340.371,230.022 334.431,227.137 328.694,224.252 323.174,221.367 317.881,218.482 312.831,215.597 308.036,212.712 
 303.51,209.827 299.268,206.942 295.324,204.057 291.691,201.172 288.384,198.287 285.414,195.402 282.796,192.517 280.539,189.631 278.655,186.746 277.152,183.861 
 276.037,180.976 275.317,178.091 274.994,175.206 275.07,172.321 275.545,169.436 276.417,166.551 277.681,163.666 279.33,160.781 281.358,157.896 283.754,155.011 
 286.508,152.126 289.608,149.241 293.042,146.356 296.795,143.471 300.855,140.586 305.207,137.701 309.837,134.816 314.731,131.931 319.875,129.046 325.257,126.161 
 330.862,123.276 336.677,120.391 342.692,117.506 348.894,114.62 355.272,111.735 361.816,108.85 368.516,105.965 375.362,103.08 382.347,100.195 389.46,97.3102 
 396.696,94.4252 404.046,91.5402 411.504,88.6551 419.063,85.7701 426.718,82.885 434.463,80 442.292,77.115 450.202,74.2299 458.186,71.3449 466.242,68.4598 
 474.365,65.5748 482.552,62.6898 490.798,59.8047 499.102,56.9197 507.459,54.0346 515.867,51.1496 524.323,48.2646 532.826,45.3795 541.372,42.4945 549.959,39.6094 
 558.585,36.7244 567.249,33.8394 575.948,30.9543 584.681,28.0693 593.446,25.1843 602.242,22.2992 611.067,19.4142 619.921,16.5291 628.801,13.6441 637.706,10.7591 
 646.636,7.87402 
 "/>
<polyline clip-path="url(#clip02)" style="stroke:#0000ff; stroke-width:1; stroke-opacity:1; fill:none" points="
 39.3701,296.378 44.1997,293.493 49.0293,290.608 53.8589,287.723 58.6886,284.838 63.5182,281.953 68.3478,279.068 73.1774,276.183 78.007,273.298 82.8367,270.413 
 87.6663,267.528 92.4959,264.643 97.3255,261.757 102.155,258.872 106.985,255.987 111.814,253.102 116.644,250.217 121.474,247.332 126.303,244.447 131.133,241.562 
 135.962,238.677 140.792,235.792 145.622,232.907 150.451,230.022 155.281,227.137 160.111,224.252 164.94,221.367 169.77,218.482 174.599,215.597 179.429,212.712 
 184.259,209.827 189.088,206.942 193.918,204.057 198.748,201.172 203.577,198.287 208.407,195.402 213.236,192.517 218.066,189.631 222.896,186.746 227.725,183.861 
 232.555,180.976 237.385,178.091 242.214,175.206 247.044,172.321 251.873,169.436 256.703,166.551 261.533,163.666 266.362,160.781 271.192,157.896 276.021,155.011 
 280.851,152.126 285.681,149.241 290.51,146.356 295.34,143.471 300.17,140.586 304.999,137.701 309.829,134.816 314.658,131.931 319.488,129.046 324.318,126.161 
 329.147,123.276 333.977,120.391 338.807,117.506 343.636,114.62 348.466,111.735 353.295,108.85 358.125,105.965 362.955,103.08 367.784,100.195 372.614,97.3102 
 377.443,94.4252 382.273,91.5402 387.103,88.6551 391.932,85.7701 396.762,82.885 401.592,80 406.421,77.115 411.251,74.2299 416.08,71.3449 420.91,68.4598 
 425.74,65.5748 430.569,62.6898 435.399,59.8047 440.229,56.9197 445.058,54.0346 449.888,51.1496 454.717,48.2646 459.547,45.3795 464.377,42.4945 469.206,39.6094 
 474.036,36.7244 478.866,33.8394 483.695,30.9543 488.525,28.0693 493.354,25.1843 498.184,22.2992 503.014,19.4142 507.843,16.5291 512.673,13.6441 517.502,10.7591 
 522.332,7.87402 
 "/>

## Examples of Tangency Portfolios

In [17]:
μb = [9; 6]/100                     #means
Σb = [ 256  0;
      0    144]/100^2
Rfb = 1/100
wT, = MVTangencyP(μb,Σb,Rfb)
printmat(wT)

wT, = MVTangencyP([13; 6]/100,Σb,Rfb)
printmat(wT)

Σb = [ 1  -0.8;
      -0.8    1]
wT, = MVTangencyP(μb,Σb,Rfb)
printmat(wT)

Σb = [ 1  0.8;
      0.8    1]
wT, = MVTangencyP(μb,Σb,Rfb)
printmat(wT)

     0.474
     0.526

     0.574
     0.426

     0.513
     0.487

     1.538
    -0.538



# Drawing the MV Frontier (Risky Assets Only) Revisisted (extra)

Recall: with only two investable assets, all portfolios of them are on the MV frontier.

We apply this idea by using the global minimum variance portfolio (see below for code) and the tangency portfolios (see above).

In [18]:
function MVMinimumVarP(μ,Σ,Rf)           #calculates the global minimum variance portfolio
    n     = length(μ)
    oneV  = ones(n,1)  
    μe    = μ - Rf                     
    Σ_1   = inv(Σ)
    w     = Σ_1*oneV/(oneV'Σ_1*oneV)[1]
    mu   = (w'μ + (1-sum(w))*Rf)[1]
    Std = sqrt((w'Σ*w)[1])
    return w,mu,Std
end

MVMinimumVarP (generic function with 1 method)

In [19]:
(wT,muT,StdT) = MVTangencyP(μ,Σ,Rf)
(wMvp,muMvp,StdMvp) = MVMinimumVarP(μ,Σ,Rf)
println("Tangency and minum variance portfolios: ")
printmat([wT wMvp])

Tangency and minum variance portfolios: 
     0.248    -0.020
     0.682     0.625
     0.070     0.395



In [20]:
v = linspace(-2.5,2.5,101)                                 #try different mixes of wT and wMvp

(ERLev,StdLev)  = (similar(v),similar(v))
for i = 1:length(v)
    w          = v[i]*wT + (1-v[i])*wMvp                        #portfolio with v[i] in wT and 1-v[i] in wMvp
    ERLev[i]   = (w'μ)[1]
    StdLev[i]  = sqrt((w'Σ*w)[1])
end


plot(StdRp*100,mustar*100,legend=nothing,color=:red,xlims=(0,15),ylims=(0,15))
plot!(StdRpRf*100,mustar*100,legend=nothing,color=:blue)
scatter!(StdLev*100,ERLev*100)
title!("Mean vs standard deviation: scatter points are new")
xlabel!("Std(Rp), %")
ylabel!("ERp, %")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.0 
 
 
 2.5 
 
 
 5.0 
 
 
 7.5 
 
 
 10.0 
 
 
 12.5 
 
 
 15.0 
 
 
 0.0 
 
 
 2.5 
 
 
 5.0 
 
 
 7.5 
 
 
 10.0 
 
 
 12.5 
 
 
 15.0 
 
 
 Mean vs standard deviation: scatter points are new 
 
 
 Std(Rp), % 
 
 
 ERp, % 
 
<polyline clip-path="url(#clip02)" style="stroke:#ff0000; stroke-width:1; stroke-opacity:1; fill:none" points="
 512.649,296.378 504.26,293.493 495.923,290.608 487.641,287.723 479.417,284.838 471.254,281.953 463.156,279.068 455.127,276.183 447.17,273.298 439.291,270.413 
 431.493,267.528 423.782,264.643 416.163,261.757 408.642,258.872 401.225,255.987 393.918,253.102 386.728,250.217 379.662,247.332 372.73,244.447 365.938,241.562 
 359.297,238.677 352.815,235.792 346.503,232.907 340.371,230.022 334.431,227.137 328.694,224.252 323.174,221.367 317.881,218.482 312.831,215.597 308.036,212.712 
 303.51,209.827 299.268,206.942 295.324,204.057 291.691,201.172 288.384,198.287 285.414,195.402 282.796,192.517 280.539,189.631 278.655,186.746 277.152,183.861 
 276.037,180.976 275.317,178.091 274.994,175.206 275.07,172.321 275.545,169.436 276.417,166.551 277.681,163.666 279.33,160.781 281.358,157.896 283.754,155.011 
 286.508,152.126 289.608,149.241 293.042,146.356 296.795,143.471 300.855,140.586 305.207,137.701 309.837,134.816 314.731,131.931 319.875,129.046 325.257,126.161 
 330.862,123.276 336.677,120.391 342.692,117.506 348.894,114.62 355.272,111.735 361.816,108.85 368.516,105.965 375.362,103.08 382.347,100.195 389.46,97.3102 
 396.696,94.4252 404.046,91.5402 411.504,88.6551 419.063,85.7701 426.718,82.885 434.463,80 442.292,77.115 450.202,74.2299 458.186,71.3449 466.242,68.4598 
 474.365,65.5748 482.552,62.6898 490.798,59.8047 499.102,56.9197 507.459,54.0346 515.867,51.1496 524.323,48.2646 532.826,45.3795 541.372,42.4945 549.959,39.6094 
 558.585,36.7244 567.249,33.8394 575.948,30.9543 584.681,28.0693 593.446,25.1843 602.242,22.2992 611.067,19.4142 619.921,16.5291 628.801,13.6441 637.706,10.7591 
 646.636,7.87402 
 "/>
<polyline clip-path="url(#clip02)" style="stroke:#0000ff; stroke-width:1; stroke-opacity:1; fill:none" points="
 39.3701,296.378 44.1997,293.493 49.0293,290.608 53.8589,287.723 58.6886,284.838 63.5182,281.953 68.3478,279.068 73.1774,276.183 78.007,273.298 82.8367,270.413 
 87.6663,267.528 92.4959,264.643 97.3255,261.757 102.155,258.872 106.985,255.987 111.814,253.102 116.644,250.217 121.474,247.332 126.303,244.447 131.133,241.562 
 135.962,238.677 140.792,235.792 145.622,232.907 150.451,230.022 155.281,227.137 160.111,224.252 164.94,221.367 169.77,218.482 174.599,215.597 179.429,212.712 
 184.259,209.827 189.088,206.942 193.918,204.057 198.748,201.172 203.577,198.287 208.407,195.402 213.236,192.517 218.066,189.631 222.896,186.746 227.725,183.861 
 232.555,180.976 237.385,178.091 242.214,175.206 247.044,172.321 251.873,169.436 256.703,166.551 261.533,163.666 266.362,160.781 271.192,157.896 276.021,155.011 
 280.851,152.126 285.681,149.241 290.51,146.356 295.34,143.471 300.17,140.586 304.999,137.701 309.829,134.816 314.658,131.931 319.488,129.046 324.318,126.161 
 329.147,123.276 333.977,120.391 338.807,117.506 343.636,114.62 348.466,111.735 353.295,108.85 358.125,105.965 362.955,103.08 367.784,100.195 372.614,97.3102 
 377.443,94.4252 382.273,91.5402 387.103,88.6551 391.932,85.7701 396.762,82.885 401.592,80 406.421,77.115 411.251,74.2299 416.08,71.3449 420.91,68.4598 
 425.74,65.5748 430.569,62.6898 435.399,59.8047 440.229,56.9197 445.058,54.0346 449.888,51.1496 454.717,48.2646 459.547,45.3795 464.377,42.4945 469.206,39.6094 
 474.036,36.7244 478.866,33.8394 483.695,30.9543 488.525,28.0693 493.354,25.1843 498.184,22.2992 503.014,19.4142 507.843,16.5291 512.673,13.6441 517.502,10.7591 
 522.332,7.87402 
 "/>